In [1]:
import pandas as pd                     
import numpy as np
import plotly.express as px
from dash import Dash,dash_table,Input,Output,html,dcc
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template as lft
import dash_bootstrap_components as dbc

Customer=pd.read_csv(r"C:\Users\akhia\Documents\Python dataanalysis\Resume project\Data\dim_customers.csv")   #load CSv files from local disk
Payment=pd.read_csv(r"C:\Users\akhia\Documents\Python dataanalysis\Resume project\Data\fact_spends.csv")

avg_spend=Payment.groupby('customer_id',as_index=False).agg({'spend':'sum'})    #Find Average spend of each customer

Customer=pd.merge(Customer,avg_spend,on='customer_id',how='left')  #add new column(spend) to table

#rename columns


Customer.rename(columns={'customer_id':'CustomerID','age_group':'AgeGroup',
                            'city':'City','occupation':'Occupation',
                             'gender':'Gender','marital status':'MaritalStatus',
                             'avg_income':'AverageIncome','spend':'AverageSpend'
                        },inplace=True)   

Payment.rename(columns={'customer_id':'CustomerID','month':'Month','category':'Category' ,
                         'payment_type':'PaymentType','spend':'Spend'
                       },inplace=True)

Customer['AvgIncomeUtilisation']=Customer['AverageSpend']/Customer['AverageIncome']   #adding new column :AVG spend utilisation


df=pd.merge(Payment,Customer,on='CustomerID',how='left')   #merging two dataframes to one


#plotly dash application

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"  #external style sheet
app=Dash(__name__,external_stylesheets=[dbc.themes.SLATE,dbc_css])
lft('SLATE')
app.layout=html.Div([
    dcc.Tabs(
    className='custom-tabs-container',
    children=[
        dcc.Tab(id='tab 0',label='Data Set',className='dbc',  #Tab 1
               children=[
                   html.H1(" Data Set in Different City",style={
                       'color':"#fa0a6a",
                       'text-align':'center',
                       'fontFamily':'Arial'
                       
                   }),
                   dbc.Row(dcc.RadioItems(id='city',           #Radioitems for different city
                                options=df['City'].unique(),
                                value='Bengaluru',className='dbc',inline=True,
                                )),
                   html.Br(),
                   dbc.Row(html.Div(id='Table') )                
                   
               ]
               ),
        dcc.Tab( id='tab 1',label='Avg income utilisation ',className='dbc',   #Tab 2
                children=[  
                            dbc.Row(
                                
                                html.H1("Average income utilisation of different catagories",
                                       style={  'color':"#fa0a6a",
                                                'text-align':'center',
                                                'fontFamily':'Arial'}
                                       )),
                        
                        html.Br(),
                            dbc.Row(dcc.RadioItems(id='check',    #Radioitems for different category
                                                   options=[ 'AgeGroup','City','Occupation','Gender','MaritalStatus'],
                                                   value='AgeGroup',
                                                   inline=True,
                                                   className='dbc')),
                        html.Br(),
                            dbc.Row(
                                  children=
                                              [dbc.Col(dcc.Graph(id='fig')),       #bar graphs in Tab 2
                                               html.Br(),
                                                dbc.Col(dcc.Graph(id='fig1-2')) ] 
                          
                                  ) 
                                
                                
                                ]),
        dcc.Tab(id='tab 2',label='Spending Insights and Payment Methods',    #tab 3
                children=[dbc.Row(html.H1("Payment Method analysis",
                                         style={  'color':"#fa0a6a",
                                                'text-align':'center',
                                                'fontFamily':'Arial'})),
                         html.Br(),
                         dbc.Row(
                                 children=[
                                     html.H2('Categories'),
                                     dbc.Col(dcc.RadioItems(id='check2',          #Radioitems for different category
                                               options=df['Category'].unique(),
                                               value=['Food'],className='dbc'),width=3),
                                     html.Br(),
                                     dbc.Col(dcc.Graph(id='fig2'),width=7),      #Line Graph
                                     dbc.Col(width=2)
                                         ]
                                 ),
                          html.Br(),
                          dbc.Row(
                                 children=[
                                     html.H2('Month'), 
                                     dbc.Col(dcc.RadioItems(id='check2-1',      #Radioitems for different month
                                               options=df['Month'].unique(),
                                               value=['May'],className='dbc')
                                             ,width=2),
                                     html.Br(),
                                     dbc.Col(dcc.Graph(id='fig2-1'),width=5),     #pie charts 
                                     dbc.Col(dcc.Graph(id='fig2-2'),width=5)
                                         ]
                                 )
                         
                         
                         ])
        
    ], colors={                             #Tab Header and style
        "border": "grey",
        "primary": "gold",
        "background": "#0f1133",
        
    },style={'fontSize':22,
             'fontWeight': 'bold'}
    )    
])

@app.callback(Output('Table','children'),             #Tab 1 Callback
              Input('city','value'))

def table_op(c):
    df_new=df.query("City == @c and PaymentType =='Credit Card'")         #dash table
    table=dash_table.DataTable(
        columns=[{"name":i,'id':i}for i in df.columns],
        data=df_new.to_dict('records'),
        filter_action='native',
        sort_action='native',
        export_format='csv',
        page_size=20,
        style_header={
            'color':'Black',
            'backgroundColor': 'lightblue',
            'fontWeight': 'bold'
        },
        style_cell_conditional=[
                {
                    'if': {'column_id': c},
                    'textAlign': 'left'
                } for c in ['CustomerId','Month']
            ],
        style_data={
            'color':'Black',
            'backgroundColor': 'lightgrey',
            
        }
#    style_as_list_view=True,
      
    )
    return table

@app.callback(Output('fig','figure'),                 #Tab 2 Callback
              Output('fig1-2','figure'),
             Input('check','value'))


def plot(v):
    
    df_new=(df.groupby( v,as_index=False)
               .agg({'AvgIncomeUtilisation':'mean'}
                ).sort_values(['AvgIncomeUtilisation'],ascending=False))
    
    fig=px.bar(df_new,                                                        #bar graph 1
                  x=v,y='AvgIncomeUtilisation', 
                   title=F"Mean Avgerage Income Spend of Different {v}"
      ).update_layout(title_x=.5,title_y=.98)
    
    fig2=px.bar((df.groupby( v,as_index=False)                         #bar graph 2
               .agg({'AverageIncome':'mean','AverageSpend':'mean'}
                ).sort_values(['AverageSpend','AverageIncome'],ascending=False)),
                  x=v,y=[ 'AverageSpend','AverageIncome'], barmode='group',
                   title=F"Income and Spend - {v}"
      ).update_layout(title_x=.5,title_y=.98)
    return fig, fig2

@app.callback(Output('fig2','figure'),                 #Tab 3 Callback
              Output('fig2-1','figure'),
              Output('fig2-2','figure'),
             Input('check2','value'),
             Input('check2-1','value'))


def plot(t,m):
    
    
    
    fig=px.line((df.groupby(['Category','Month','PaymentType'],as_index=False).agg({'Spend':'sum'})     #line graph 
                .query("Category == @t")),
         x='Month',y='Spend',
        color="PaymentType",
        title="Different Payment Methods"
       ).update_layout(title_x=.5,title_y=.95)
    
    fig2=px.pie(((df.groupby(['Month','PaymentType'],as_index=False).agg({'Spend':'sum'})  #pie graph 1
                .query("Month == @m"))),
       values='Spend',names='PaymentType'
               ).update_layout(
        title=f'Different Payment Methods {m} month',
        title_x=.5,title_y=.95)
    fig3=px.pie(((df.groupby(['Month','Category'],as_index=False).agg({'Spend':'sum'})  #pie graph 2
                .query("Month  == @m"))),
       values='Spend',names='Category').update_layout(
        title=f'Different Categories {m} month',
        title_x=.5,title_y=.95)
    
    return fig  ,fig2,fig3
    
if __name__=='__main__':           #running application
    app.run_server(
        jupyter_mode='tab',
        port=8338)

Dash app running on http://127.0.0.1:8338/


<IPython.core.display.Javascript object>